In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output

In [2]:
sub_path = "Submissions"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
0,5941774d,2.138313e-02,0.010011,0.003657,0.016455,0.220130
1,4023181e,6.183937e-01,0.404991,0.020802,0.986631,0.014379
2,b20200e4,9.098931e-07,0.007189,0.001704,0.001559,0.000041
3,e7f018bb,9.738217e-01,0.997395,0.999148,0.999509,0.992729
4,4371c8c3,4.257437e-01,0.067517,0.001759,0.997644,0.074289


In [3]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
is_iceberg_0,1.000000,0.703407,0.630337,0.474636,0.601516
is_iceberg_1,0.703407,1.000000,0.834851,0.532091,0.843965
is_iceberg_2,0.630337,0.834851,1.000000,0.435538,0.842089
is_iceberg_3,0.474636,0.532091,0.435538,1.000000,0.409355
is_iceberg_4,0.601516,0.843965,0.842089,0.409355,1.000000


In [4]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [5]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [6]:
# load the model with best base performance
sub_base = pd.read_csv('Submissions/submission4_best.csv')

In [7]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')